# Temperature - Basic Usage of HiGlass

In this notebook we will talk through the main aspects of the API and how to get started visualizing data

In [1]:
%load_ext autoreload
%autoreload 2

### Load data

In [14]:
from utils import get_ncei_temperature_data_as_array

temps_by_city = get_ncei_temperature_data_as_array({
    "boston": {
        "files": [
            "data/1577451.csv.gz",
            "data/1577468.csv.gz",
            "data/1573178.csv.gz"
        ],
        "station": 72509014739
    },
    "fairbanks": {
        "files": ["data/1707246.csv.gz"],
        "station": 70261026411
    }
})

### Create tilesets

In [20]:
from higlass.tilesets import Tileset
from clodius.tiles.npvector import tileset_info, tiles_wrapper

tilesets = {}
for city in temps_by_city:
    tilesets[city] = Tileset(
        tileset_info=lambda: tileset_info(temps_by_city[city][0]),
        tiles=lambda tile_ids: tiles_wrapper(temps_by_city[city][0], tile_ids, temps_by_city[city][1])
    )

### Create HiGlass view

In [21]:
import higlass
import higlass.tilesets as hgt
from higlass.client import Track, CombinedTrack, View


top_axis = Track('top-axis')
boston_line = Track(
    'horizontal-line',
    tileset=tilesets['boston'],
    height=100,
    options={
        'name': 'Boston',
        'lineStrokeColor': 'green',
        'lineStrokeWidth': 2,
        'constIndicators': [0],
    },
)
fairbanks_line = Track(
    'horizontal-line',
    tileset=tilesets['fairbanks'],
    height=100,
    options={
        'name': 'Fairbanks',
        'lineStrokeColor': 'blue',
        'lineStrokeWidth': 2,
        'constIndicators': [0],
    },
)

display, server, viewconf = higlass.display([
    View([top_axis, boston_line, fairbanks_line])
])

display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'T48vFa9XQxaa0gjSOjKyNA', 'tracks': {'top': [{'ty…